# Starbucks | Data Playground
### 2019 July 10
`Cameron.Cn`
{&#9982;}

---
##### Sites for Use
+ https://www.dataquest.io/blog/jupyter-notebook-tips-tricks-shortcuts/
+ https://medium.com/@plotlygraphs/introducing-plotly-express-808df010143d

---

In [ ]:
# beginning off by importing our libraries, as per usual
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns
import statsmodels.formula.api as smf
import plotly_express as px

from datetime import datetime
from sklearn.linear_model import LogisticRegression
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import classification_report
#from sklearn.metrics import confusion_matrix
#from sklearn.naive_bayes import GaussianNB   #import Gaussian Bayes modeling function
#from sklearn.metrics import accuracy_score
#from sklearn.naive_bayes import BernoulliNB

%matplotlib inline
print("All Imported Successfully!")

In [2]:
# to change (more pretty) the standard plt output
#sns.set(rc={'figure.figsize':(11.7,8.27)})
#sns.reset_orig()

In [79]:
px.__version__

'0.3.1'

In [5]:
loc = ("coffee_stock_cleaned.csv")
cofe = pd.read_csv(loc)
print(cofe.head(),
      '\n-----\n',
      cofe.tail())

         date   close    volume  coffeePrice
0  1992-06-29  0.3594  58732800       0.5774
1  1992-06-30  0.3477  34777600       0.5760
2  1992-07-01  0.3555  18316800       0.5940
3  1992-07-02  0.3555  13996800       0.6000
4  1992-07-06  0.3555   5753600       0.5930 
-----
             date  close   volume  coffeePrice
6762  2019-06-28  83.83  6735620       1.0921
6763  2019-07-01  84.55  7940539       1.1135
6764  2019-07-02  85.51  6206087       1.0965
6765  2019-07-03  87.79  7818552       1.1365
6766  2019-07-05  87.79  5733239       1.1110


In [9]:
revdi = ("starbucks_revised_directory.csv")
stloc = pd.read_csv(revdi)
stloc.head()

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude,Coordinates
0,Starbucks,47370-257954,"Meritxell, 96",Licensed,"Av. Meritxell, 96",Andorra la Vella,7,AD,AD500,376818720,GMT+1:00 Europe/Andorra,1.53,42.51,POINT (1.53 42.51)
1,Starbucks,22331-212325,Ajman Drive Thru,Licensed,"1 Street 69, Al Jarf",Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.42,POINT (55.47 25.42)
2,Starbucks,47089-256771,Dana Mall,Licensed,Sheikh Khalifa Bin Zayed St.,Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.39,POINT (55.47 25.39)
3,Starbucks,22126-218024,Twofour 54,Licensed,Al Salam Street,Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.38,24.48,POINT (54.38 24.48)
4,Starbucks,17127-178586,Al Ain Tower,Licensed,"Khaldiya Area, Abu Dhabi Island",Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.54,24.51,POINT (54.54 24.51)


In [8]:
px.scatter(cofe, x="date", y='close', size='volume', color='coffeePrice', size_max=30)

In [24]:
#making an interactive map chart (commented out because it slows down the notebook
# far too much). There are the following projections available:
 #          ['equirectangular', 'mercator', 'orthographic', 'natural
 #           earth', 'kavrayskiy7', 'miller', 'robinson', 'eckert4',
 #           'azimuthal equal area', 'azimuthal equidistant', 'conic
 #           equal area', 'conic conformal', 'conic equidistant',
 #           'gnomonic', 'stereographic', 'mollweide', 'hammer',
 #           'transverse mercator', 'albers usa', 'winkel tripel',
 #           'aitoff', 'sinusoidal']
# But no Dymaxion?! 
    
    
#px.scatter_geo(stloc,lat='Latitude',lon='Longitude',projection="gasdfn")

In [43]:
stcount = stloc.groupby('Country').count()
stcount.head()

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Postcode,Phone Number,Timezone,Longitude,Latitude,Coordinates
Country,,,,,,,,,,,,,
AD,1,1,1,1,1,1,1,1,1,1,1,1,1
AE,144,144,144,144,144,144,144,24,78,144,144,144,144
AR,108,108,108,108,108,108,108,100,29,108,108,108,108
AT,18,18,18,18,18,18,18,18,17,18,18,18,18
AU,22,22,22,22,22,22,22,22,0,22,22,22,22


In [73]:
stcount2 = stcount.drop(['Store Number','Store Name','Ownership Type',
                              'Street Address','City','State/Province',
                              'Postcode','Phone Number','Timezone',
                              'Longitude','Latitude','Coordinates'], axis=1)
stcount2.head()

,Brand
Country,
AD,1
AE,144
AR,108
AT,18
AU,22


In [74]:
# we lost our index in the above maneuvers, so here we are getting it back
# so that both Country and Brand can be called later
stcount2 = stcount2.reset_index()
stcount2.head()

,Country,Brand
0,AD,1
1,AE,144
2,AR,108
3,AT,18
4,AU,22


In [153]:
# our data is in ISO-2 when we need ISO-3 country names.
# Ugh. We will have to convert them all!
# https://github.com/gsnaveen/plotly-worldmap-mapping-2letter-CountryCode-to-3letter-country-code


conv = pd.read_csv('../countryMap.txt')
#
conv.head()

,Countrylet,2let,3let
0,Afghanistan,AF,AFG
1,Aland Islands,AX,ALA
2,Albania,AL,ALB
3,Algeria,DZ,DZA
4,American Samoa,AS,ASM


In [154]:
conv.drop(['Countrylet'], axis=1, inplace=True)
conv.head()

,2let,3let
0,AF,AFG
1,AX,ALA
2,AL,ALB
3,DZ,DZA
4,AS,ASM


In [155]:
conv['Country'] = conv['2let']

In [156]:
conv.head()

,2let,3let,Country
0,AF,AFG,AF
1,AX,ALA,AX
2,AL,ALB,AL
3,DZ,DZA,DZ
4,AS,ASM,AS


In [157]:
l3t = pd.merge(conv,
              stcount2[['Country','Brand']],
              on='Country')
l3t.head()

,2let,3let,Country,Brand
0,AD,AND,AD,1
1,AR,ARG,AR,108
2,AW,ABW,AW,3
3,AU,AUS,AU,22
4,AT,AUT,AT,18


In [158]:
l33t = l3t.drop(columns=['2let','Country'])
l33t.head()

,3let,Brand
0,AND,1
1,ARG,108
2,ABW,3
3,AUS,22
4,AUT,18


In [159]:
l33t = l33t.rename(columns={'Brand':'Stores'})
l33t.head()

,3let,Stores
0,AND,1
1,ARG,108
2,ABW,3
3,AUS,22
4,AUT,18


In [160]:
px.choropleth(l33t, locations='3let', color='Stores',
              color_continuous_scale=px.colors.sequential.Viridis)

In [161]:
px.choropleth(l33t, locations='3let', color='Stores',
              color_continuous_scale=px.colors.sequential.Plasma)